This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'awAtVUSk51eDrf4wAWr5'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import json 
import operator
import numpy as np

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-01-01&api_key=' + API_KEY
r = requests.get(url)
r_json = r.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_json

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-12-01T14:48:09.907Z',
  'newest_available_date': '2020-12-01',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-01',
  'end_date': '2017-01-01',
  'data': [],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [9]:
#1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
r = requests.get(url)

In [10]:
#2.Convert the returned JSON object into a Python dictionary.
r_json = r.json()
type(r_json)

dict

In [12]:
#Calculate what the highest and lowest opening prices were for the stock in this period.
#Review the data content
r_json.keys()

dict_keys(['dataset'])

In [13]:
r_json['dataset'].keys()

dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])

In [14]:
r_json['dataset']['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [15]:
r_json['dataset']['data'][0:2]

[['2017-12-29',
  51.76,
  51.94,
  51.45,
  51.76,
  None,
  34640.0,
  1792304.0,
  None,
  None,
  None],
 ['2017-12-28',
  51.65,
  51.82,
  51.43,
  51.6,
  None,
  40660.0,
  2099024.0,
  None,
  None,
  None]]

In [25]:
#3.Calculate what the highest and lowest opening prices were for the stock in this period.
d_json = r_json['dataset']['data']
opening = [i[1] for i in d_json if i[1] != None]
print('highest_open:'+str(max(opening)))
print('lowest_open:'+str(min(opening)))

highest_open:53.11
lowest_open:34.0


In [26]:
#4.What was the largest change in any one day (based on High and Low price).
high = [i[2] for i in d_json if i[2] != None]
low = [i[3] for i in d_json if i[3] != None]
change = [abs(j - k) for (j, k) in zip(high, low)]
print ("largest change in any one day:" + str(max(change)))

largest change in any one day:2.8100000000000023


In [28]:
#5.What was the largest change between any two days (based on Closing Price).
close = [i[4] for i in d_json if i[4] != None]
close_next = [i[4] for i in d_json if i[4] != None][1:]
change = [abs(j - k) for (j, k) in zip(close, close_next)]
print ("largest change between two days:" + str(max(change)))

largest change between two days:2.559999999999995


In [29]:
#6.What was the average daily trading volume during this year?
trading_volume = [row[6] for row in data_json]
volume_avg = sum(trading_volume) / len(trading_volume)
print ("The avarage daily trading volume in 2017: " + str(round(volume_avg,2)))

The avarage daily trading volume in 2017: 89124.34


In [ ]:
#7.(Optional) What was the median trading volume during this year.
def median(x):
    sorted_trading_vol = sorted(x, reverse= False)
    size = len(sorted_trading_vol)
    if size % 2 == 1:
        return sorted_trading_vol[size//2]
    else:
        return sum(sorted_trading_vol[size//2-1:size//2+1])/2.0    
print('\n',find_median(values = trading_volume) , ' is the median trading volume during this year')